Portfolio
=========
The Portfolio object stands between the Strategy and the Risk. It is responsible for pulling the orders from the associated
strategies and then passing the orders to the Risk. If the strategy is using Intents it will turn intents into orders and also
does any internal crossing.

In [1]:
import data as datalib
from collections import namedtuple
from tomahawk.strategy import ExampleStrategy
from tomahawk import portfolio, OrderManager, PositionManager


In [2]:
# setup database credentials
db_credentials = credentials('test', 'localhost', prefix='db_')

In [3]:
# setup the market data
csvdf = datalib.CsvDataFeed("c:/rmbaries/git/pifq/src/puma/data/tests/inst/csv_data_feed", source_name='test_csv_data_feed')
csvdh = datalib.HistoricalDataManager(csvdf, **db_credentials)
csvdl = datalib.LiveDataManager(csvdf, **db_credentials)
mdm = datalib.MarketDataManager(csvdh, csvdl)

In [5]:
# initialize the Portfolio and OrderManager objects, then feed them to the Strategy object
oms = OrderManager('test_unit', None)
pm = PositionManager('test_unit', oms, None)
port = portfolio.Portfolio('port_test', oms, pm)
port.setup_market_data(live_frequency='1min', market_data_manager=mdm)
obj_bridge = namedtuple('OB', 'order_manager, market_data_manager')(oms, mdm)
strat = ExampleStrategy('TEST1', obj_bridge)

In [6]:
# Attach the Strategy object binds to the Portfolio

port.add_strategy(strat)
print(port.strategy_ids)
print(strat.portfolio)

['TEST1']


In [7]:
# insert symbols into strategy
strat.add_symbols([('stock', 'test.sym.1', '1min'), ('stock', 'test.sym.2', '1D')])

In [8]:
# create some orders and intents in the strategy

strat.order('stock', 'test.sym.1', 'B', 100, 'LIMIT', 100.5)
strat.order('stock', 'test.sym.2', 'S', 55, 'LIMIT', 5.5)
strat.intent('stock', 'test.sym.1', -60)

# see the orders in OMS, note that the intents are not turned into orders yet
print(oms.open_orders_df())

  index  fill_quantity    Order__uuid                           create_timestamp                    quantity  fill_price    exchange_order_id    broker_order_id    symbol      buy_sell    originator_uuid                       originator_id    commission    strategy_uuid                         product_type    booked    portfolio_id    CREATED                           state    closed    type    details           portfolio_uuid    event_type    strategy_id
-------  ---------------  ------------------------------------  --------------------------------  ----------  ------------  -------------------  -----------------  ----------  ----------  ------------------------------------  ---------------  ------------  ------------------------------------  --------------  --------  --------------  --------------------------------  -------  --------  ------  ----------------  ----------------  ------------  -------------
      0                   b81808d1-527e-4b45-a617-38b1c8eecdc6  2017-03-24 04:

In [9]:
# setup the MarketDataManager
mdm.bartime = '2010-01-01 09:31:00'
mdm.update('stock', '1min')

In [10]:
# Now process the orders with the Portfolio object
port.process_orders()

In [11]:
# now that the process_orders has been called the intent has been turned into an order
print(oms.open_orders_df({'originator_uuid': port.uuid}))

  index  fill_quantity    Order__uuid                           create_timestamp                    quantity  fill_price    exchange_order_id    broker_order_id    symbol      buy_sell    originator_uuid                       originator_id        commission    strategy_uuid                         product_type    booked    portfolio_id    CREATED                           state    closed    type    STAGED                            details            portfolio_uuid                        event_type    strategy_id
-------  ---------------  ------------------------------------  --------------------------------  ----------  ------------  -------------------  -----------------  ----------  ----------  ------------------------------------  -------------------  ------------  ------------------------------------  --------------  --------  --------------  --------------------------------  -------  --------  ------  --------------------------------  -----------------  --------------------------

In [12]:
# see that there are no orders left in CREATED state
oms.orders_list({'state': 'CREATED'})

[]

In [13]:
# they are now in STAGED state
oms.orders_list({'state': 'STAGED'})

[tomahawk.order.Order : UUID=b81808d1-527e-4b45-a617-38b1c8eecdc6,
 tomahawk.order.Order : UUID=7e12d1d1-a4df-407d-b50a-7d569cf29cad,
 tomahawk.order.Order : UUID=5d42e617-6a55-49b8-b1a6-704a2f9fccda]

In [14]:
# inspect the first order in the staged order list
oms.orders_list()[0].print()

uuid         : b81808d1-527e-4b45-a617-38b1c8eecdc6
state        : STAGED
originator_id: strategy.TEST1
strategy_id  : TEST1
symbol       : test.sym.1
buy/sell     : buy
quantity     : 100
order_type   : LIMIT
details      : {'price': 100.5}
broker_id    : None
exchange_id  : None
fill_price   : None
fill_qty     : None
commission   : None
booked       : None
closed       : False

state history:
  index  timestamp                         state
-------  --------------------------------  -------
      0  2017-03-24 04:33:49.834074+00:00  CREATED
      1  2017-03-24 04:33:56.271594+00:00  STAGED

replacements :
  index    quantity  details
-------  ----------  ----------------
      0         100  {'price': 100.5}

fills        :
index    timestamp    bartime    quantity    price    commission    booked
-------  -----------  ---------  ----------  -------  ------------  --------
